# 1. Not optimized

In [2]:
import pandas as pd
df = pd.read_csv(
    'household_power_consumption.zip', decimal= '.', sep = ';', header=0, compression = 'zip'
)


D:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (2,3,4,5,6,7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
df.head()

,Date,Time,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
0,16/12/2006,17:24:00,4.216,0.418,234.840,18.400,0.000,1.000,17.0
1,16/12/2006,17:25:00,5.360,0.436,233.630,23.000,0.000,1.000,16.0
2,16/12/2006,17:26:00,5.374,0.498,233.290,23.000,0.000,2.000,17.0
3,16/12/2006,17:27:00,5.388,0.502,233.740,23.000,0.000,1.000,17.0
4,16/12/2006,17:28:00,3.666,0.528,235.680,15.800,0.000,1.000,17.0


In [4]:
import os
statinfo = os.stat('household_power_consumption.zip');
statinfo.st_size / 1024 / 1024

18.384346961975098

In [5]:
df.info(memory_usage = 'deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075259 entries, 0 to 2075258
Data columns (total 9 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   Date                   object 
 1   Time                   object 
 2   Global_active_power    object 
 3   Global_reactive_power  object 
 4   Voltage                object 
 5   Global_intensity       object 
 6   Sub_metering_1         object 
 7   Sub_metering_2         object 
 8   Sub_metering_3         float64
dtypes: float64(1), object(8)
memory usage: 1007.3 MB


# 2. Use proper bit representation, remove nulls, parse dates

In [6]:
df.dropna(axis=0, how='any', thresh=None, subset=None, inplace=True)
df.to_csv('electric-power-consumption-data-set-cleaned.csv')

In [7]:
import numpy as np
dateparse_3 = lambda x: pd.to_datetime(x, format = '%d/%m/%Y %H:%M:%S')

df = pd.read_csv(
    'electric-power-consumption-data-set-cleaned.csv',
    decimal= '.',
    sep = ',',
    header=0,
    dtype = {'Global_active_power':np.float32, 'Global_reactive_power':np.float32, 'Voltage':np.float64 ,
         'Global_intensity':np.float32, 'Sub_metering_1':np.uint8, 'Sub_metering_2':np.uint8, 
         'Sub_metering_3':np.uint8},
    date_parser = dateparse_3, parse_dates = {'Date_time':['Date', 'Time']}
)


In [8]:
df.info(memory_usage = 'deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2049280 entries, 0 to 2049279
Data columns (total 9 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   Date_time              datetime64[ns]
 1   Unnamed: 0             int64         
 2   Global_active_power    float32       
 3   Global_reactive_power  float32       
 4   Voltage                float64       
 5   Global_intensity       float32       
 6   Sub_metering_1         uint8         
 7   Sub_metering_2         uint8         
 8   Sub_metering_3         uint8         
dtypes: datetime64[ns](1), float32(3), float64(1), int64(1), uint8(3)
memory usage: 76.2 MB


In [9]:
print(len(df.index))

2049280


# 3. Use iterator

In [10]:
df = pd.read_csv(
    'electric-power-consumption-data-set-cleaned.csv',
    decimal= '.',
    sep = ',',
    header=0,
    dtype = {'Global_active_power':np.float32, 'Global_reactive_power':np.float32, 'Voltage':np.float64 ,
         'Global_intensity':np.float32, 'Sub_metering_1':np.uint8, 'Sub_metering_2':np.uint8, 
         'Sub_metering_3':np.uint8},
    date_parser = dateparse_3, parse_dates = {'Date_time':['Date', 'Time']},
    iterator=True
)

df.get_chunk(1000) 

,Date_time,Unnamed: 0,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
0,2006-12-16 17:24:00,0,4.216,0.418,234.84,18.4,0,1,17
1,2006-12-16 17:25:00,1,5.360,0.436,233.63,23.0,0,1,16
2,2006-12-16 17:26:00,2,5.374,0.498,233.29,23.0,0,2,17
3,2006-12-16 17:27:00,3,5.388,0.502,233.74,23.0,0,1,17
4,2006-12-16 17:28:00,4,3.666,0.528,235.68,15.8,0,1,17
...,...,...,...,...,...,...,...,...,...
995,2006-12-17 09:59:00,995,2.296,0.054,238.13,9.6,0,0,0
996,2006-12-17 10:00:00,996,2.292,0.054,238.03,9.6,0,0,0
997,2006-12-17 10:01:00,997,0.370,0.000,240.26,2.4,0,0,0
998,2006-12-17 10:02:00,998,0.472,0.000,240.15,2.4,0,0,0


In [11]:
chunks = pd.read_csv(
    'electric-power-consumption-data-set-cleaned.csv',
    decimal= '.',
    sep = ',',
    header=0,
    dtype = {'Global_active_power':np.float32, 'Global_reactive_power':np.float32, 'Voltage':np.float64 ,
         'Global_intensity':np.float32, 'Sub_metering_1':np.uint8, 'Sub_metering_2':np.uint8, 
         'Sub_metering_3':np.uint8},
    date_parser = dateparse_3, parse_dates = {'Date_time':['Date', 'Time']},
    chunksize=10000
)

 
for chunk in chunks:
   print('Shape:', chunk.shape)
   print(chunk.head(),'\n') 

Shape: (10000, 9)
            Date_time  Unnamed: 0  Global_active_power  Global_reactive_power  \
0 2006-12-16 17:24:00           0                4.216                  0.418   
1 2006-12-16 17:25:00           1                5.360                  0.436   
2 2006-12-16 17:26:00           2                5.374                  0.498   
3 2006-12-16 17:27:00           3                5.388                  0.502   
4 2006-12-16 17:28:00           4                3.666                  0.528   

   Voltage  Global_intensity  Sub_metering_1  Sub_metering_2  Sub_metering_3  
0   234.84              18.4               0               1              17  
1   233.63              23.0               0               1              16  
2   233.29              23.0               0               2              17  
3   233.74              23.0               0               1              17  
4   235.68              15.8               0               1              17   

Shape: (10000, 9)
 

Shape: (10000, 9)
                Date_time  Unnamed: 0  Global_active_power  \
80000 2007-02-10 06:50:00       80006                1.338   
80001 2007-02-10 06:51:00       80007                1.340   
80002 2007-02-10 06:52:00       80008                1.418   
80003 2007-02-10 06:53:00       80009                1.396   
80004 2007-02-10 06:54:00       80010                1.402   

       Global_reactive_power  Voltage  Global_intensity  Sub_metering_1  \
80000                  0.000   241.56               5.4               0   
80001                  0.000   241.83               5.4               0   
80002                  0.062   241.42               5.8               0   
80003                  0.070   240.18               5.8               0   
80004                  0.072   240.57               5.8               0   

       Sub_metering_2  Sub_metering_3  
80000               0              18  
80001               0              18  
80002               0              18 

Shape: (10000, 9)
                 Date_time  Unnamed: 0  Global_active_power  \
160000 2007-04-06 20:13:00      160009                0.326   
160001 2007-04-06 20:14:00      160010                0.346   
160002 2007-04-06 20:15:00      160011                0.324   
160003 2007-04-06 20:16:00      160012                0.342   
160004 2007-04-06 20:17:00      160013                0.320   

        Global_reactive_power  Voltage  Global_intensity  Sub_metering_1  \
160000                  0.070   242.92               1.4               0   
160001                  0.078   243.60               1.4               0   
160002                  0.072   243.09               1.4               0   
160003                  0.074   242.41               1.4               0   
160004                  0.068   242.21               1.4               0   

        Sub_metering_2  Sub_metering_3  
160000               0               0  
160001               0               0  
160002               0 

Shape: (10000, 9)
                 Date_time  Unnamed: 0  Global_active_power  \
240000 2007-06-03 23:37:00      243733                1.390   
240001 2007-06-03 23:38:00      243734                2.332   
240002 2007-06-03 23:39:00      243735                1.392   
240003 2007-06-03 23:40:00      243736                2.286   
240004 2007-06-03 23:41:00      243737                0.816   

        Global_reactive_power  Voltage  Global_intensity  Sub_metering_1  \
240000                  0.250   238.31               5.8               0   
240001                  0.254   237.52              10.0               0   
240002                  0.252   238.61               5.8               0   
240003                  0.256   237.77               9.8               0   
240004                  0.264   239.40               3.8               0   

        Sub_metering_2  Sub_metering_3  
240000              19               0  
240001              33               0  
240002              19 

Shape: (10000, 9)
                 Date_time  Unnamed: 0  Global_active_power  \
320000 2007-07-29 15:50:00      323906                0.194   
320001 2007-07-29 15:51:00      323907                0.194   
320002 2007-07-29 15:52:00      323908                0.192   
320003 2007-07-29 15:53:00      323909                0.192   
320004 2007-07-29 15:54:00      323910                0.104   

        Global_reactive_power  Voltage  Global_intensity  Sub_metering_1  \
320000                  0.118   235.69               1.0               0   
320001                  0.120   235.85               1.0               0   
320002                  0.118   235.59               1.0               0   
320003                  0.118   235.46               1.0               0   
320004                  0.000   235.47               0.4               0   

        Sub_metering_2  Sub_metering_3  
320000               0               0  
320001               0               0  
320002               0 

...

Shape: (10000, 9)
                  Date_time  Unnamed: 0  Global_active_power  \
1680000 2010-03-05 08:47:00     1691483                0.410   
1680001 2010-03-05 08:48:00     1691484                0.414   
1680002 2010-03-05 08:49:00     1691485                0.414   
1680003 2010-03-05 08:50:00     1691486                0.414   
1680004 2010-03-05 08:51:00     1691487                0.406   

         Global_reactive_power  Voltage  Global_intensity  Sub_metering_1  \
1680000                  0.116   246.47               1.8               0   
1680001                  0.118   246.65               1.8               0   
1680002                  0.118   247.04               1.8               0   
1680003                  0.120   247.21               1.8               0   
1680004                  0.116   246.84               1.8               0   

         Sub_metering_2  Sub_metering_3  
1680000               0               0  
1680001               0               1  
1680002 

Shape: (10000, 9)
                  Date_time  Unnamed: 0  Global_active_power  \
1760000 2010-05-01 07:55:00     1773511                0.376   
1760001 2010-05-01 07:56:00     1773512                0.374   
1760002 2010-05-01 07:57:00     1773513                0.372   
1760003 2010-05-01 07:58:00     1773514                0.370   
1760004 2010-05-01 07:59:00     1773515                0.370   

         Global_reactive_power  Voltage  Global_intensity  Sub_metering_1  \
1760000                  0.182   239.50               1.6               0   
1760001                  0.182   239.13               1.6               0   
1760002                  0.180   238.85               1.6               0   
1760003                  0.178   238.12               1.6               0   
1760004                  0.178   238.18               1.6               0   

         Sub_metering_2  Sub_metering_3  
1760000               1               1  
1760001               1               1  
1760002 

Shape: (10000, 9)
                  Date_time  Unnamed: 0  Global_active_power  \
1840000 2010-06-25 21:17:00     1853513                0.972   
1840001 2010-06-25 21:18:00     1853514                1.122   
1840002 2010-06-25 21:19:00     1853515                0.840   
1840003 2010-06-25 21:20:00     1853516                0.774   
1840004 2010-06-25 21:21:00     1853517                0.780   

         Global_reactive_power  Voltage  Global_intensity  Sub_metering_1  \
1840000                  0.374   240.66               5.0               0   
1840001                  0.378   240.51               5.2               0   
1840002                  0.348   240.91               3.8               0   
1840003                  0.334   240.98               3.4               0   
1840004                  0.332   241.08               3.4               0   

         Sub_metering_2  Sub_metering_3  
1840000               2               0  
1840001               1               1  
1840002 

Shape: (10000, 9)
                  Date_time  Unnamed: 0  Global_active_power  \
1920000 2010-08-25 11:05:00     1940741                0.172   
1920001 2010-08-25 11:06:00     1940742                0.172   
1920002 2010-08-25 11:07:00     1940743                0.174   
1920003 2010-08-25 11:08:00     1940744                0.172   
1920004 2010-08-25 11:09:00     1940745                0.172   

         Global_reactive_power  Voltage  Global_intensity  Sub_metering_1  \
1920000                    0.0   240.40               0.8               0   
1920001                    0.0   240.50               0.8               0   
1920002                    0.0   240.81               0.8               0   
1920003                    0.0   240.48               0.8               0   
1920004                    0.0   240.02               0.8               0   

         Sub_metering_2  Sub_metering_3  
1920000               0               0  
1920001               0               1  
1920002 

Shape: (10000, 9)
                  Date_time  Unnamed: 0  Global_active_power  \
2000000 2010-10-23 15:42:00     2025978                3.510   
2000001 2010-10-23 15:43:00     2025979                3.782   
2000002 2010-10-23 15:44:00     2025980                3.692   
2000003 2010-10-23 15:45:00     2025981                2.714   
2000004 2010-10-23 15:46:00     2025982                2.484   

         Global_reactive_power  Voltage  Global_intensity  Sub_metering_1  \
2000000                  0.240   244.51              15.2              28   
2000001                  0.294   243.88              16.0              34   
2000002                  0.344   243.73              15.4              32   
2000003                  0.350   244.86              11.8              16   
2000004                  0.350   245.57              10.8              12   

         Sub_metering_2  Sub_metering_3  
2000000               1              19  
2000001               2              19  
2000002 

# 4. Use sqlite

In [12]:
from sqlalchemy import create_engine
database = create_engine('sqlite:///csv_database.db')
 
chunks = pd.read_csv(
    'electric-power-consumption-data-set-cleaned.csv',
    decimal= '.',
    sep = ',',
    header=0,
    dtype = {'Global_active_power':np.float32, 'Global_reactive_power':np.float32, 'Voltage':np.float64 ,
         'Global_intensity':np.float32, 'Sub_metering_1':np.uint8, 'Sub_metering_2':np.uint8, 
         'Sub_metering_3':np.uint8},
    date_parser = dateparse_3, parse_dates = {'Date_time':['Date', 'Time']},
    chunksize=10000
)
 
for chunk in chunks:
    chunk.to_sql('power_consumption', database, if_exists='append')
 
pd.read_sql_query('SELECT * FROM power_consumption limit 0,10', database)

,index,Date_time,Unnamed: 0,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
0,0,2006-12-16 17:24:00.000000,0,4.216,0.418,234.84,18.4,0,1,17
1,1,2006-12-16 17:25:00.000000,1,5.360,0.436,233.63,23.0,0,1,16
2,2,2006-12-16 17:26:00.000000,2,5.374,0.498,233.29,23.0,0,2,17
3,3,2006-12-16 17:27:00.000000,3,5.388,0.502,233.74,23.0,0,1,17
4,4,2006-12-16 17:28:00.000000,4,3.666,0.528,235.68,15.8,0,1,17
5,5,2006-12-16 17:29:00.000000,5,3.520,0.522,235.02,15.0,0,2,17
6,6,2006-12-16 17:30:00.000000,6,3.702,0.520,235.09,15.8,0,1,17
7,7,2006-12-16 17:31:00.000000,7,3.700,0.520,235.22,15.8,0,1,17
8,8,2006-12-16 17:32:00.000000,8,3.668,0.510,233.99,15.8,0,1,17
9,9,2006-12-16 17:33:00.000000,9,3.662,0.510,233.86,15.8,0,2,16
